In [17]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec



In [13]:
print(os.getenv("PINECONE_API_KEY"))

f5aef84f-b159-4962-a942-ccc61d4f2929


In [15]:
pc = Pinecone(api_key = os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

In [16]:
import json
data = json.load(open("reviews.json"))
data["reviews"]

[{'professor': 'Dr. Alice Johnson',
  'subject': 'Introduction to Psychology',
  'stars': 5,
  'review': 'Dr. Johnson is an outstanding professor with a passion for the subject. Her lectures are engaging and she makes complex topics easy to understand.'},
 {'professor': 'Prof. Mark Davis',
  'subject': 'Calculus I',
  'stars': 4,
  'review': 'Prof. Davis is knowledgeable and helpful, though his lectures can sometimes be a bit fast-paced.'},
 {'professor': 'Dr. Emily Chen',
  'subject': 'Organic Chemistry',
  'stars': 3,
  'review': "Dr. Chen's classes are informative, but the material can be challenging. A lot of self-study is required."},
 {'professor': 'Prof. Samuel Brown',
  'subject': 'History of Art',
  'stars': 5,
  'review': 'Prof. Brown brings history to life with his enthusiasm and deep knowledge of the subject. Highly recommend.'},
 {'professor': 'Dr. Laura Smith',
  'subject': 'Microeconomics',
  'stars': 4,
  'review': 'Dr. Smith is a good professor who explains economic co

In [21]:
proccessed_data = []
client = OpenAI()

# semantic embedding
for review in data['reviews']:
    response = client.embeddings.create(
        input = review['review'],
        model = "text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    proccessed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })


In [22]:
proccessed_data[0]

{'values': [0.0064648152,
  0.0020447203,
  0.00608523,
  0.033474695,
  0.008208537,
  -0.004815991,
  0.014969903,
  0.055372033,
  -0.0053290245,
  -0.004190268,
  0.03634531,
  -0.00411613,
  -0.0353489,
  0.033688214,
  -0.002410961,
  0.023107266,
  -0.03518283,
  0.0029788564,
  0.040995233,
  0.050674662,
  0.045526534,
  -0.025408505,
  0.029275531,
  -0.014424249,
  -0.02521871,
  -0.036938414,
  0.023000509,
  0.016618727,
  0.032644354,
  -0.019263964,
  0.08218026,
  0.010604669,
  -0.01825569,
  -0.013700664,
  -0.01489873,
  0.01373625,
  0.002106996,
  0.009240534,
  0.00852288,
  0.02335637,
  0.0020150652,
  -0.006061506,
  -0.02375968,
  0.008255985,
  0.04289316,
  0.0235343,
  0.005435783,
  -0.013226182,
  0.04787522,
  0.05262004,
  -0.029773736,
  0.0002720857,
  0.042489853,
  -0.02486285,
  -0.037080757,
  0.01077667,
  0.018991137,
  0.03755524,
  -0.006108954,
  -0.03539635,
  0.034921866,
  -0.003941165,
  -0.031908907,
  -0.008048398,
  -0.012870321,
  -0.

In [23]:
index = pc.Index('rag')
index.upsert(
    vectors=proccessed_data,
    namespace="ns1"
)


{'upserted_count': 21}

In [24]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 21}},
 'total_vector_count': 21}